In [1]:
!pip install raganything


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.5/90.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.6/786.6 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

In [3]:
# Verify installation
!mineru --version

# Check if properly configured
!python -c "from raganything import RAGAnything; rag = RAGAnything(); print('✅ MinerU installed properly' if rag.check_parser_installation() else '❌ MinerU installation issue')"

mineru, version 2.7.6
INFO: RAGAnything initialized with config:
INFO:   Working directory: ./rag_storage
INFO:   Parser: mineru
INFO:   Parse method: auto
INFO:   Multimodal processing - Image: True, Table: True, Equation: True
INFO:   Max concurrent files: 1
✅ MinerU installed properly


In [1]:
!rm -rf ./rag_storage/*

In [2]:
import shutil

shutil.rmtree("output", ignore_errors=True)
shutil.rmtree("rag_storage", ignore_errors=True)

print("Deleted successfully")

Deleted successfully


In [1]:
import asyncio
import os
import shutil
import logging
import nest_asyncio
import numpy as np
from openai import AsyncOpenAI
from raganything import RAGAnything, RAGAnythingConfig
from lightrag.llm.openai import openai_complete_if_cache
from lightrag.utils import EmbeddingFunc

# 1. CẤU HÌNH MÔI TRƯỜNG
nest_asyncio.apply()
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("RAGAnything-Fix")

async def main():
    # --- THÔNG TIN CẤU HÌNH ---
    # API key bạn đã cung cấp
    api_key = "YOUR_API"
    working_dir = "./rag_storage"
    input_file = "/content/2510.12323v1.pdf"
    output_dir = "./output"

    # --- DỌN DẸP DỮ LIỆU CŨ ---
    if os.path.exists(working_dir):
        logger.info(f"Đang xóa dữ liệu cũ tại {working_dir}...")
        shutil.rmtree(working_dir)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # --- 2. CẤU HÌNH RAGANYTHING ---
    config = RAGAnythingConfig(
        working_dir=working_dir,
        parser="mineru",
        parse_method="auto",
        enable_image_processing=True,
        enable_table_processing=True,
        enable_equation_processing=True,
    )

    aclient = AsyncOpenAI(api_key=api_key)

    # --- 3. ĐỊNH NGHĨA CÁC HÀM XỬ LÝ (MODELS) ---
    async def llm_model_func(prompt, system_prompt=None, history_messages=[], **kwargs):
        return await openai_complete_if_cache(
            "gpt-4o-mini",
            prompt,
            system_prompt=system_prompt,
            history_messages=history_messages,
            api_key=api_key,
            **kwargs,
        )

    async def vision_model_func(
        prompt, system_prompt=None, history_messages=[], image_data=None, messages=None, **kwargs
    ):
        if messages:
            return await openai_complete_if_cache(
                "gpt-4o", "", messages=messages, api_key=api_key, **kwargs
            )
        elif image_data:
            return await openai_complete_if_cache(
                "gpt-4o", "",
                messages=[
                    {"role": "system", "content": system_prompt} if system_prompt else None,
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": prompt},
                            {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_data}"}},
                        ],
                    }
                ],
                api_key=api_key, **kwargs
            )
        else:
            return await llm_model_func(prompt, system_prompt, history_messages, **kwargs)

    # --- HÀM EMBEDDING (FIX TRẢ VỀ NUMPY ARRAY) ---
    async def wrapped_embedding_func(texts: list[str]) -> np.ndarray:
        response = await aclient.embeddings.create(
            model="text-embedding-3-large",
            input=texts
        )

        embeddings = [item.embedding for item in response.data]

        if len(embeddings) == len(texts) * 2:
            embeddings = embeddings[:len(texts)]

        # CHUYỂN ĐỔI SANG NUMPY ARRAY ĐỂ TRÁNH LỖI 'size'
        return np.array(embeddings)

    embedding_func = EmbeddingFunc(
        embedding_dim=3072,
        max_token_size=8192,
        func=wrapped_embedding_func
    )

    # --- 4. KHỞI TẠO VÀ CHẠY RAG ---
    rag = RAGAnything(
        config=config,
        llm_model_func=llm_model_func,
        vision_model_func=vision_model_func,
        embedding_func=embedding_func,
    )

    logger.info("Bắt đầu parse và index tài liệu (Quá trình này có thể mất vài phút)...")
    try:
        await rag.process_document_complete(
            file_path=input_file,
            output_dir=output_dir,
            parse_method="auto"
        )
        logger.info("Xử lý tài liệu hoàn tất!")
    except Exception as e:
        logger.error(f"Lỗi trong quá trình xử lý: {e}")
        return

    # --- 5. KIỂM TRA KẾT QUẢ ---
    print("\n" + "="*50)
    print("TRUY VẤN THỬ NGHIỆM")
    print("="*50)

    try:
        text_result = await rag.aquery(
            "Tóm tắt các phát hiện chính trong tài liệu này?",
            mode="hybrid"
        )
        print(f"\n[Kết quả văn bản]:\n{text_result}")

        multimodal_result = await rag.aquery_with_multimodal(
            "Giải thích ý nghĩa của công thức này trong ngữ cảnh tài liệu",
            multimodal_content=[{
                "type": "equation",
                "latex": "P(d|q) = \\frac{P(q|d) \\cdot P(d)}{P(q)}",
                "equation_caption": "Relevance probability"
            }],
            mode="hybrid"
        )
        print(f"\n[Kết quả đa phương thức]:\n{multimodal_result}")
    except Exception as e:
        print(f"\n[Lỗi truy vấn]: {e}")

if __name__ == "__main__":
    asyncio.run(main()) if not asyncio.get_event_loop().is_running() else await main()

INFO: Created working directory: ./rag_storage
INFO: RAGAnything initialized with config:
INFO:   Working directory: ./rag_storage
INFO:   Parser: mineru
INFO:   Parse method: auto
INFO:   Multimodal processing - Image: True, Table: True, Equation: True
INFO:   Max concurrent files: 1
INFO: Parser 'mineru' installation verified
INFO: Initializing LightRAG with parameters: {'working_dir': './rag_storage'}
INFO: [] Created new empty graph file: ./rag_storage/graph_chunk_entity_relation.graphml
INFO: [] Process 32769 KV load full_docs with 0 records
INFO: [] Process 32769 KV load text_chunks with 0 records
INFO: [] Process 32769 KV load full_entities with 0 records
INFO: [] Process 32769 KV load full_relations with 0 records
INFO: [] Process 32769 KV load entity_chunks with 0 records
INFO: [] Process 32769 KV load relation_chunks with 0 records
INFO: [] Process 32769 KV load llm_response_cache with 0 records
INFO: [] Process 32769 doc status load doc_status with 0 records
INFO: [] Process


TRUY VẤN THỬ NGHIỆM


INFO:  == LLM cache == saving: hybrid:keywords:6751a2c83f9cff3cd5b7554954a4155f
INFO: Query nodes: Phát hiện chính, Tài liệu (top_k:40, cosine:0.2)
INFO: Local query: 40 entites, 100 relations
INFO: Query edges: Summary, Key findings, Document analysis (top_k:40, cosine:0.2)
INFO: Global query: 52 entites, 40 relations
INFO: Raw search results: 85 entities, 130 relations, 0 vector chunks
INFO: After truncation: 85 entities, 130 relations
INFO: Selecting 60 from 60 entity-related chunks by vector similarity
INFO: Find no additional relations-related chunks from 130 relations
INFO: Round-robin merged chunks: 60 -> 60 (deduplicated 0)
INFO: Final context: 85 entities, 130 relations, 20 chunks
INFO: Final chunks S+F/O: E2/1 E1/2 E2/3 E2/4 E3/5 E1/6 E2/7 E3/8 E2/9 E4/10 E2/11 E3/12 E5/13 E1/14 E1/15 E3/16 E2/17 E2/18 E6/19 E1/20
INFO: Found 3 image path matches in prompt
INFO: Processed 3 images for VLM
INFO: VLM enhanced query completed
INFO: Executing multimodal query: Giải thích ý nghĩa 


[Kết quả văn bản]:
Tài liệu này tập trung vào việc giới thiệu và phân tích một khung xử lý toàn diện có tên "RAG-Anything: All-In-One RAG Framework". Dưới đây là một số phát hiện chính từ tài liệu:

1. **Khung Xử Lý RAG-Anything**: Được giới thiệu như một giải pháp toàn diện cho việc tạo dữ liệu bổ sung từ các nguồn khác nhau (Retrieval-Augmented Generation - RAG), nhấn mạnh vào việc tích hợp đa phương thức để cải thiện khả năng truy xuất và hiểu dữ liệu. Khung này nổi bật với chiến lược dựng đồ thị kép và cơ chế truy xuất kết hợp, giúp xử lý các tài liệu phức tạp hơn so với các phương pháp truyền thống [1].

2. **Hiệu Suất Cao Trong Các Ngữ Cảnh Phức Tạp**: RAG-Anything đã chứng minh khả năng vượt trội trong việc xử lý các tài liệu dài và đa phương thức. Nó đạt được độ chính xác cao nhất trong bộ dữ liệu DocBench với 63.4%, và hiệu suất nổi bật trong việc xử lý các báo cáo nghiên cứu và báo cáo tài chính, cho thấy khả năng lý giải và tích hợp dữ liệu đa dạng [1].

3. **Kết Cấu và Khả

INFO: Multimodal query content processing completed
INFO: Generated enhanced query length: 2495 characters
INFO: Executing VLM enhanced query: User query: Giải thích ý nghĩa của công thức này trong ngữ cảnh tài liệu

Related equation content: ...
INFO:  == LLM cache == saving: hybrid:keywords:d5c8cc6cebffa00e455c747912dd3a05
INFO: Query nodes: Posterior probability, Likelihood, Prior probability, Marginal likelihood, Medical Diagnosis, Spam Detection, Machine Learning, Finance, Inference, Decision Making, Uncertainty, Bayesian Statistics (top_k:40, cosine:0.2)
INFO: Local query: 40 entites, 158 relations
INFO: Query edges: Bayes' Theorem, Probability theory, Statistical significance (top_k:40, cosine:0.2)
INFO: Global query: 54 entites, 40 relations
INFO: Raw search results: 81 entities, 183 relations, 0 vector chunks
INFO: After truncation: 81 entities, 176 relations
INFO: Selecting 69 from 69 entity-related chunks by vector similarity
INFO: Find no additional relations-related chunks


[Lỗi truy vấn]: RetryError[<Future at 0x7dda7b647950 state=finished raised RateLimitError>]


In [4]:
import asyncio
import os
import logging
import nest_asyncio
import numpy as np
from openai import AsyncOpenAI
from raganything import RAGAnything, RAGAnythingConfig
from lightrag.llm.openai import openai_complete_if_cache
from lightrag.utils import EmbeddingFunc

# 1. CẤU HÌNH MÔI TRƯỜNG
nest_asyncio.apply()
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("RAG-Query")

async def run_query_loop():
    # --- THÔNG TIN CẤU HÌNH ---
    # Lưu ý: Thay API Key mới nếu bạn đã thu hồi key cũ để bảo mật
    api_key = "YOUR_API"
    working_dir = "./rag_storage"

    if not os.path.exists(working_dir):
        print(f"Lỗi: Không tìm thấy thư mục {working_dir}. Bạn cần chạy block Index tài liệu trước!")
        return

    # 2. CẤU HÌNH RAGANYTHING
    config = RAGAnythingConfig(
        working_dir=working_dir,
        parser="mineru",
        enable_image_processing=True,
        enable_table_processing=True,
        enable_equation_processing=True,
    )

    aclient = AsyncOpenAI(api_key=api_key)

    # 3. ĐỊNH NGHĨA CÁC HÀM XỬ LÝ (Bắt buộc phải có đủ)
    async def llm_model_func(prompt, system_prompt=None, history_messages=[], **kwargs):
        return await openai_complete_if_cache(
            "gpt-4o-mini",
            prompt,
            system_prompt=system_prompt,
            history_messages=history_messages,
            api_key=api_key,
            **kwargs,
        )

    # Thêm vision_model để hỗ trợ mode hybrid/multimodal
    async def vision_model_func(prompt, system_prompt=None, history_messages=[], **kwargs):
        return await llm_model_func(prompt, system_prompt, history_messages, **kwargs)

    async def wrapped_embedding_func(texts: list[str]) -> np.ndarray:
        response = await aclient.embeddings.create(
            model="text-embedding-3-large",
            input=texts
        )
        return np.array([item.embedding for item in response.data])

    embedding_func = EmbeddingFunc(
        embedding_dim=3072,
        max_token_size=8192,
        func=wrapped_embedding_func
    )

    # 4. KHỞI TẠO VÀ KÍCH HOẠT ENGINE
    rag = RAGAnything(
        config=config,
        llm_model_func=llm_model_func,
        vision_model_func=vision_model_func, # Cần thiết cho các phiên bản mới
        embedding_func=embedding_func,
    )

    # --- BƯỚC QUAN TRỌNG: Nạp dữ liệu cũ ---
    try:
        await rag._ensure_lightrag_initialized()
        print("\n✅ Đã kết nối với dữ liệu đã index thành công!")
    except Exception as e:
        print(f"⚠️ Cảnh báo khi khởi tạo database: {e}")

    # 5. VÒNG LẶP HỎI ĐÁP
    print("\n" + "="*50)
    print("SẴN SÀNG TRUY VẤN. Gõ 'exit' để thoát.")
    print("="*50)

    # Sử dụng loop để tránh treo khi dùng input() trong async
    loop = asyncio.get_event_loop()

    while True:
        query = await loop.run_in_executor(None, input, "\nCâu hỏi của bạn: ")

        if query.lower() in ['exit', 'quit', 'thoát']:
            print("Đang đóng ứng dụng...")
            break
        if not query.strip():
            continue

        try:
            print("🤖 Đang suy nghĩ...")
            # Sử dụng mode 'hybrid' để tìm cả trong văn bản và quan hệ thực thể
            result = await rag.aquery(query, mode="hybrid")
            print(f"\n[Trả lời]:\n{result}")
        except Exception as e:
            print(f"❌ Lỗi truy vấn: {e}")

# Chạy mã
if __name__ == "__main__":
    if asyncio.get_event_loop().is_running():
        # Nếu đang chạy trong Colab/Jupyter
        await run_query_loop()
    else:
        # Nếu chạy file .py bình thường
        asyncio.run(run_query_loop())

INFO: RAGAnything initialized with config:
INFO:   Working directory: ./rag_storage
INFO:   Parser: mineru
INFO:   Parse method: auto
INFO:   Multimodal processing - Image: True, Table: True, Equation: True
INFO:   Max concurrent files: 1
INFO: Parser 'mineru' installation verified
INFO: Initializing LightRAG with parameters: {'working_dir': './rag_storage'}
INFO: [] Loaded graph from ./rag_storage/graph_chunk_entity_relation.graphml with 678 nodes, 1307 edges
INFO: [] Process 46164 KV load full_docs with 1 records
INFO: [] Process 46164 KV load text_chunks with 91 records
INFO: [] Process 46164 KV load full_entities with 1 records
INFO: [] Process 46164 KV load full_relations with 1 records
INFO: [] Process 46164 KV load entity_chunks with 107 records
INFO: [] Process 46164 KV load relation_chunks with 75 records
INFO: [] Process 46164 KV load llm_response_cache with 192 records
INFO: [] Process 46164 doc status load doc_status with 1 records
INFO: [] Process 46164 KV load parse_cache


✅ Đã kết nối với dữ liệu đã index thành công!

SẴN SÀNG TRUY VẤN. Gõ 'exit' để thoát.

Câu hỏi của bạn: paper này là gì


INFO: Executing VLM enhanced query: paper này là gì...
INFO: LLM func: 4 new workers initialized (Timeouts: Func: 180s, Worker: 360s, Health Check: 375s)


🤖 Đang suy nghĩ...


INFO:  == LLM cache == saving: hybrid:keywords:8615a8274c67dde474234665aec22a90
INFO: Embedding func: 8 new workers initialized (Timeouts: Func: 30s, Worker: 60s, Health Check: 75s)
INFO: Query nodes: N/A (top_k:40, cosine:0.2)
INFO: Local query: 40 entites, 154 relations
INFO: Query edges: Paper, Definition, Purpose (top_k:40, cosine:0.2)
INFO: Global query: 52 entites, 40 relations
INFO: Raw search results: 90 entities, 191 relations, 0 vector chunks
INFO: After truncation: 90 entities, 160 relations
INFO: Selecting 78 from 78 entity-related chunks by vector similarity
INFO: Find no additional relations-related chunks from 160 relations
INFO: Round-robin merged chunks: 78 -> 78 (deduplicated 0)
INFO: Final context: 90 entities, 160 relations, 20 chunks
INFO: Final chunks S+F/O: E2/1 E4/2 E5/3 E2/4 E2/5 E3/6 E2/7 E1/8 E4/9 E2/10 E1/11 E1/12 E4/13 E3/14 E4/15 E4/16 E3/17 E4/18 E3/19 E2/20
INFO: Found 6 image path matches in prompt
INFO: Processed 6 images for VLM
INFO: VLM enhanced que

...


[Trả lời]:
### Bảng So Sánh Các Phương Pháp

Dưới đây là bảng so sánh về hiệu suất của các phương pháp khác nhau khi xử lý dữ liệu trên tập dữ liệu DocBench. Bảng này tập trung vào các phương pháp cụ thể và chỉ ra độ chính xác của chúng qua nhiều miền và thể loại tài liệu khác nhau.

| **Phương Pháp**        | **Miền**        | **Aca.** | **Fin.** | **Gov.** | **Law.** | **News** | **Tổng Quan** |
|------------------------|------------------|----------|----------|----------|----------|----------|--------------|
| **Chunk-only**         |                  | 55.8     | 61.5     | 60.1     | 60.7     | 64.0     | 60.0         |
| **w/o Reranker**       |                  | 60.9     | 63.5     | 58.8     | 60.2     | 68.6     | 62.4         |
| **RAGAnything**        |                  | 61.4     | 67.0     | 61.5     | 60.2     | 66.3     | **63.4**     |

#### Phân Tích Bảng
- **RAGAnything** đạt độ chính xác cao nhất tổng thể (63.4%), cho thấy khả năng xử lý dữ liệu của nó vượt trội hơ

INFO: Executing VLM enhanced query: cho tôi các hình ảnh...


🤖 Đang suy nghĩ...


INFO:  == LLM cache == saving: hybrid:keywords:f203e9abd6e9bf2e40d911a2a7e5d200
INFO: Query nodes: Hình ảnh (top_k:40, cosine:0.2)
INFO: Local query: 24 entites, 118 relations
INFO: Query edges: Hình ảnh (top_k:40, cosine:0.2)
INFO: Global query: 43 entites, 40 relations
INFO: Raw search results: 50 entities, 122 relations, 0 vector chunks
INFO: After truncation: 50 entities, 122 relations
INFO: Selecting 48 from 48 entity-related chunks by vector similarity
INFO: Find no additional relations-related chunks from 122 relations
INFO: Round-robin merged chunks: 48 -> 48 (deduplicated 0)
INFO: Final context: 50 entities, 122 relations, 20 chunks
INFO: Final chunks S+F/O: E6/1 E3/2 E3/3 E2/4 E4/5 E4/6 E3/7 E4/8 E4/9 E1/10 E3/11 E4/12 E5/13 E4/14 E3/15 E5/16 E5/17 E4/18 E3/19 E5/20
INFO: Found 19 image path matches in prompt
INFO: Processed 19 images for VLM
INFO: VLM enhanced query completed



[Trả lời]:
Dựa trên thông tin từ ngữ liệu, dưới đây là các hình ảnh cùng với thông tin mô tả tương ứng:

1. **Hình ảnh: RAG-Anything**
   - Mô tả: Trình bày quy trình sử dụng các lớp LSTM hai chiều để tạo nhãn đầu ra từ các ký tự đầu vào.
   - **Hình path**: `/content/output/2510.12323v1/hybrid_auto/images/ae0d073fec8e9fc5912602c472ad1d217648e1c9e8a1305093aea7dadfed02bf.jpg`

2. **Hình ảnh: GPT-4o-mini**
   - Mô tả: Mô hình BERT-to-BERT đạt được giá trị AUPRC cao nhất trong dataset Evidence Inference.
   - **Hình path**: `/content/output/2510.12323v1/hybrid_auto/images/09dbbdbce919902a905ccde7512a58d9f9e781dce78bed4abf2678859dfa506f.jpg`

3. **Hình ảnh: LightRAG**
   - Mô tả: Tăng cường độ chính xác trong lĩnh vực "Train" bằng phương pháp fine-tuning GEM.
   - **Hình path**: `/content/output/2510.12323v1/hybrid_auto/images/e463aea5b04288ae18185bd159ad748512f9137f6d84994db730778cc5eaa5c8.jpg`

4. **Hình ảnh: MMGraphRAG**
   - Mô tả: Mô hình "Bert-To-Bert" đạt giá trị AUPRC là 0.708 tro